-----------------------------------------------------
Created on: 03-09-2022                              
Author: Rohit Sharma                                
                                                     
-----------------------------------------------------
# Example of PySpark ML

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [3]:
spark

In [4]:
# read the dataset
training = spark.read.csv("test5.csv", header=True, inferSchema=True)

In [5]:
training.show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|    Ram| 31|         4|  1000|
|  Rohan| 20|         3| 30000|
|  Mohan| 25|         1| 40000|
|    Raj| 21|         5| 20000|
|   Neha| 22|         2| 20000|
|Prateek| 26|         6| 50000|
|  Mukul| 24|         8| 30000|
+-------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

-------------------------------------------------------------------------------------------------------------------------------
Usually in ml algos in sklearn we do train test split but in pyspark we have to create a way where we group all the independent features <br>

   **[Age,Experience] -----> new feature ------> independent feature** <br>

In order to group this in Pyspark we use VectoreAssembler from pyspark.ml.feature

--------------------------------------------------------------------------------------------------------------------------------

In [8]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["Age","Experience"], outputCol="Independent Feature")

In [9]:
output = assembler.transform(training)

In [10]:
output.show()

+-------+---+----------+------+-------------------+
|   Name|Age|Experience|Salary|Independent Feature|
+-------+---+----------+------+-------------------+
|    Ram| 31|         4|  1000|         [31.0,4.0]|
|  Rohan| 20|         3| 30000|         [20.0,3.0]|
|  Mohan| 25|         1| 40000|         [25.0,1.0]|
|    Raj| 21|         5| 20000|         [21.0,5.0]|
|   Neha| 22|         2| 20000|         [22.0,2.0]|
|Prateek| 26|         6| 50000|         [26.0,6.0]|
|  Mukul| 24|         8| 30000|         [24.0,8.0]|
+-------+---+----------+------+-------------------+



In [11]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Feature']

In [14]:
finalized_data = output.select("Independent Feature", "Salary")

In [15]:
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|         [31.0,4.0]|  1000|
|         [20.0,3.0]| 30000|
|         [25.0,1.0]| 40000|
|         [21.0,5.0]| 20000|
|         [22.0,2.0]| 20000|
|         [26.0,6.0]| 50000|
|         [24.0,8.0]| 30000|
+-------------------+------+



Now here **Independent Feature** column is independent feature and **Salary** is dependent feature column

In [16]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])

In [17]:
regressor = LinearRegression(featuresCol='Independent Feature', labelCol='Salary')
regressor = regressor.fit(train_data)

In [18]:
# coefficients
regressor.coefficients

DenseVector([-839.3574, 6317.2691])

In [19]:
# Interepts
regressor.intercept

10329.317269076635

In [21]:
# Prediction
pred_result = regressor.evaluate(test_data)

In [22]:
pred_result.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [20.0,3.0]| 30000|12493.975903614542|
|         [22.0,2.0]| 20000| 4497.991967871554|
|         [25.0,1.0]| 40000|-4337.349397590317|
+-------------------+------+------------------+



In [23]:
pred_result.meanAbsoluteError, pred_result.meanSquaredError

(25781.793842034745, 837524561.4318017)